### Creating function which are needed.

- Function to find all which meet condition, first and second but also specifiy the place.
- Some form of way to print the frequecy of each. This could just use the itertools.

In [2]:
from methods import LMEO, Static
lm = LMEO()

In [13]:
# Filter function.
def filter_perm_in_pos(all_ranks, first_hotel_id, second_hotel_id, pos0 = 0):
    '''This filters by a combo i, j or j, i starting at pos0'''
    i, j = pos0, pos0 + 1
    
    def create_condition(hotel_id_obj, place):
        if isinstance(hotel_id_obj, list):
            return lambda rank: any([hotel_id in rank[place] for hotel_id in hotel_id_obj])
        else:
            return lambda rank: hotel_id_obj in rank[place]
    
    first_in_zero = create_condition(first_hotel_id, i)
    second_in_one = create_condition(second_hotel_id, j)
    
    second_in_zero = create_condition(second_hotel_id, i)
    first_in_one = create_condition(first_hotel_id, j)
    
    first_perm_list = []
    second_perm_list = []
    neither_perm_list = []

    for rank in all_ranks:
        # This logic could be better but I dont really care.
        if first_in_zero(rank) and second_in_one(rank):
            first_perm_list.append(rank)
        elif second_in_zero(rank) and first_in_one(rank):
            second_perm_list.append(rank)
        else:
            neither_perm_list.append(rank)

    return first_perm_list, second_perm_list, neither_perm_list


In [14]:
# This function will print the number of occurences for the combo in pos 0
import itertools

def count_perm_combos(rank_data, target_words, pos0, min = 50):
    word_pair_combinations = list(itertools.combinations(target_words, 2))

    for word1, word2 in word_pair_combinations:
        first, second, _ = filter_perm_in_pos(rank_data, word1, word2, pos0)
        if len(first) < min or len(second) < min:
            continue
        print(word1, word2, len(first), len(second))


### Loading the relevant data.

In [15]:
London3000 = lm.order_set('', 'Bristol1001', 0) # lazy because it has the load in it.
London_words = 'Savoy Langham Ritz Clarid hangri Dorchest Shard Rose Royal Ned Corin Mandarin'.split()

Loading Previously Generated Ranks.


### The Experiment. 

We will first identifiy the combos which are most sensible to test. We will require that both there needs to be over 50 in each position. 

In [30]:
# Identifiying the relevant combos. 
min = 100
for i in range(0, 3):
    print(i)
    count_perm_combos(London3000, London_words, i, min)

0
Savoy Ritz 174 849
1
Savoy Langham 252 365
Langham Dorchest 178 107
2
Langham Clarid 206 174


We are testing whether the next item in the percent dict is distributed in the same way for each of the combos. 

if pos0 != 0 then we need to consider that the first item cannot appear twice, so we have an aditional assumption that the dist of the zeoth doesnt affect the fourth. 

To do:
Filter as before then get the correct percent dicts and then devise a method to compare the two. There should be a hypothesis test here but you dont know much about them at all.

In [31]:
import numpy as np

word_pair_combinations = list(itertools.combinations(London_words, 2))

for pos in range(0, 3):
    for word1, word2 in word_pair_combinations:
        first, second, neither = filter_perm_in_pos(London3000, word1, word2, pos)
        
        if len(first) <= min or len(second) <= min:
            continue
                
        first = np.column_stack(first)
        second = np.column_stack(second)

        first_dict = lm.create_percent_dict(first, London_words, 1)
        second_dict = lm.create_percent_dict(second, London_words, 1)

        # How should you properly compare the percents.
        
        residuals = [value - second_dict[pos+2].get(key, 0) for key, value in first_dict[pos+2].items()]
        mean = np.mean(residuals)
        std = np.std(residuals)        

        print(f'Comparing the combo {word1}, {word2} in position {pos}')
        print(f'The average difference between the keys is {mean} with std: {std} \n')


Comparing the combo Savoy, Ritz in position 0
The average difference between the keys is 0.0010600706713778196 with std: 0.044103053876124586 

Comparing the combo Savoy, Langham in position 1
The average difference between the keys is 0.0005479452054794268 with std: 0.02947337764737162 

Comparing the combo Langham, Dorchest in position 1
The average difference between the keys is 9.71445146547012e-17 with std: 0.022613727628819136 

Comparing the combo Langham, Clarid in position 2
The average difference between the keys is 0.0021551724137930527 with std: 0.027671023808658383 



The average difference between the keys is 0.
    - So it is not bias as in the two rankings arent bias in one direction for the combo
    - The Std is realtively low but not unconsequential. 

I am unsure if I conclude that the previous order doesnt effect the first choice from here. 